In [1]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt

In [66]:
from google.colab import files
uploaded = files.upload()

Saving sansotei_google_review_2.csv to sansotei_google_review_2.csv


In [82]:
processed_data = []
x = ''
data = csv.reader(open("sansotei_google_review_2.csv", encoding='utf-8'))
for i, line in enumerate(data):
    x = line[0].split(";")
    # print(x)
    y = [x[3], x[5]]
    # print(y)
    processed_data.append(y)

In [107]:
def split_content(content):
    # separate punctuations
    content = content.lower().replace(".", "") \
                 .replace(",", "") \
                 .replace(";", "") \
                 .replace("?", "") \
                 .replace("the", "") \
                 .replace("and", "")
    return content.split()

In [91]:
import torchtext
glove = torchtext.vocab.GloVe(name="6B", dim=50)

In [108]:
for i, line in enumerate(processed_data):
    if i > 30:
        break
    print(sum(int(w in glove.stoi) for w in split_content(line[-1])))

1
7
2
1
24
17
11
28
3
2
15
35
18
28
2
37
12
36
15
19
31
15
20
6
9
14
33
17
35
1
8


In [105]:
def get_content_words(glove_vector):
    train, valid, test = [], [], []
    for i, line in enumerate(processed_data):
            content = line[-1]
            idxs = [glove_vector.stoi[w]        # lookup the index of word
                    for w in split_content(content)
                    if w in glove_vector.stoi] # keep words that has an embedding
            if not idxs: # ignore tweets without any word with an embedding
                continue
            idxs = torch.tensor(idxs) # convert list to pytorch tensor
            label = torch.tensor(int(line[0] >= "3")).long()
            if i % 5 < 3:
                train.append((idxs, label))
            elif i % 5 == 4:
                valid.append((idxs, label))
            else:
                test.append((idxs, label))
    return train, valid, test

train, valid, test = get_content_words(glove)

In [106]:
print(len(train))
print(len(valid))
print(len(test))

265
88
88


In [101]:
content, label = train[0]
print(content.shape)
print(label)

torch.Size([1])
tensor(1)
